---

## Set Up PostgreSQL (on a Mac)

Install with Homebrew:

```
$ brew update && brew install postgres
```

Or, if already installed,

```
$ brew update && brew upgrade postgres
```

Run `initdb` just once, basically to create the directory structure and such on disk that's needed for creating new databases. Note: The specified path should match the version of Postgres just installed!

```
$ initdb /usr/local/var/postgres9.5 -E utf8
```

To _manually_ start and stop a local Postgres server from running, use

```
$ pg_ctl -D /usr/local/var/postgres -l /usr/local/var/postgres/server.log start
$ pg_ctl -D /usr/local/var/postgres stop -s -m fast
```
        
Or to _automatically_ start a Postgres server (now and) at launch:

```
$ mkdir -p ~/Library/LaunchAgents
$ ln -sfv /usr/local/opt/postgresql/*.plist ~/Library/LaunchAgents
$ launchctl load ~/Library/LaunchAgents/homebrew.mxcl.postgresql.plist
```

Open the system paths file, `/etc/paths`, in a text editor, and move the line `/usr/local/bin` from the bottom of the file to the top (if it wasn't like this already). If you had to make a change to the file, reboot the computer. After rebooting, the command `which psql` command should return `/usr/local/bin/psql`.

Homebrew automatically created a database superuser account with the same login as your current Mac OS account. Let's create a dedicated user named `app` for connecting to and owning the app's database:

```
$ createuser --echo --pwprompt --superuser --createdb app
```

You'll be prompted to create a password — be sure to remember it or save it somewhere!

Homebrew also automatically created a database named `postgres` that may be used to log info for administrative tasks such as creating a user. Let's create a new database for this project:

```
$ createdb --echo --encoding=utf8 --host=<HOST> --port=<PORT> --username=app --owner=app <DBNAME>
```

We're using our `app` user to create the database, and assign it as the db's owner. **TODO:** Choose a `<DBNAME>` for this project!

To access the database through an interactive shell:

```
$ psql --host=<HOST> --port=<PORT> --username=app --dbname=<DBNAME>
```

Lastly, define a `DATABASE_URL` environment variable containing this same information so that programs such as those in `cipy` can access it:

```
$ export DATABASE_URL=postgres://app<PASSWORD>@<HOST>/<DBNAME>
```

---

## Create Citations Table from Templates and DDL

In [6]:
from __future__ import unicode_literals

from getpass import getpass
import io

import psycopg2
import yaml

In [7]:
with io.open('../cipy/db/statement_templates.yaml', mode='rt') as f:
    templates = yaml.load(f)
    
templates

{'create_table': 'CREATE TABLE IF NOT EXISTS {table_name} ({columns})',
 'drop_table': 'DROP TABLE IF EXISTS {table_name}',
 'insert': 'INSERT INTO {table_name}({columns}) VALUES ({values})'}

In [12]:
with io.open('../cipy/db/ddls/citations.yaml', mode='rt') as f:
    ddl = yaml.load(f)

ddl

{'columns': [{'name': 'record_id', 'type': 'BIGSERIAL NOT NULL'},
  {'name': 'project_id', 'type': 'INTEGER NOT NULL'},
  {'name': 'user_id', 'type': 'INTEGER NOT NULL'},
  {'name': 'insert_ts', 'type': 'TIMESTAMP(0) WITHOUT TIME ZONE NOT NULL'},
  {'name': 'type_of_work', 'type': 'VARCHAR(25)'},
  {'name': 'title', 'type': 'VARCHAR(250)'},
  {'name': 'secondary_title', 'type': 'VARCHAR(250)'},
  {'name': 'publication_year', 'type': 'SMALLINT'},
  {'name': 'publication_month', 'type': 'SMALLINT'},
  {'name': 'authors', 'type': 'VARCHAR(100) ARRAY'},
  {'name': 'abstract', 'type': 'TEXT'},
  {'name': 'keywords', 'type': 'VARCHAR(100) ARRAY'},
  {'name': 'type_of_reference', 'type': 'VARCHAR(50)'},
  {'name': 'journal_name', 'type': 'VARCHAR(100)'},
  {'name': 'volume', 'type': 'VARCHAR(20)'},
  {'name': 'issue_number', 'type': 'VARCHAR(20)'},
  {'name': 'doi', 'type': 'VARCHAR(100)'},
  {'name': 'issn', 'type': 'VARCHAR(20)'},
  {'name': 'publisher', 'type': 'VARCHAR(100)'},
  {'name': 

In [21]:
create_table_stmt = templates['create_table'].format(
    table_name=citations_ddl['table_name'],
    columns=', '.join(column['name'] + ' ' + column['type']
                      for column in citations_ddl['columns']))
create_table_stmt

'CREATE TABLE IF NOT EXISTS citations (record_id BIGSERIAL NOT NULL, project_id INTEGER NOT NULL, user_id INTEGER NOT NULL, insert_ts TIMESTAMP(0) WITHOUT TIME ZONE NOT NULL, type_of_work VARCHAR(25), title VARCHAR(250), secondary_title VARCHAR(250), publication_year SMALLINT, publication_month SMALLINT, authors VARCHAR(100) ARRAY, abstract TEXT, keywords VARCHAR(100) ARRAY, type_of_reference VARCHAR(50), journal_name VARCHAR(100), volume VARCHAR(20), issue_number VARCHAR(20), doi VARCHAR(100), issn VARCHAR(20), publisher VARCHAR(100), language VARCHAR(50), other_fields JSONB, is_duplicate BOOLEAN)'

In [22]:
conn = psycopg2.connect(database='appname', user='app', host='localhost', port=5432)#, password=getpass())

In [23]:
with conn.cursor() as cur:
    cur.execute(create_table_stmt)
    conn.commit()

In [32]:
templates['insert'].format(
    table_name=citations_ddl['table_name'],
    columns=,
    values=)

KeyError: 'columns'